In [13]:
from google.colab import userdata


Bad news: iNaturalist's API didn't make available their CV module. However, Plant.id has the same feature and it is much easier to use.
We can use Plant.id to identify the plant.
https://www.kindwise.com/plant-id

They also have another feature that we can use, called "Plant Health":
https://www.kindwise.com/plant-health

Then, we can combine both features and pass it on to the LLM.

# Getting Plant ID from a picture

In [15]:
import requests
import base64

# Define the endpoint and API key
url = "https://api.plant.id/v2/identify"
api_key = userdata.get('plantid_key')  # Replace with your actual API key

# Load the image and encode it in base64
image_path = "plant.jpg"
with open(image_path, "rb") as image_file:
    base64_image = base64.b64encode(image_file.read()).decode("utf-8")

# Prepare the headers and data payload
headers = {
    "Content-Type": "application/json",
    "Api-Key": api_key
}

data = {
    "images": [base64_image],
    "organs": ["leaf"],  # Specify which part of the plant you are identifying (leaf, flower, etc.)
    "include-related-images": False,  # Optional: Include related images in response
    "include-plant-details": ["common_names", "url", "wiki_description"]  # Specify the plant details you need
}

# Send POST request
response = requests.post(url, headers=headers, json=data)

# Check the response
if response.status_code == 200:
    results = response.json()
    print("Possible plant identifications:")
    for suggestion in results["suggestions"]:
        print(f"Species: {suggestion['plant_name']}, Probability: {suggestion['probability']*100:.2f}%")
        # Additional details (if requested)
        if "common_names" in suggestion:
            print("Common Names:", ", ".join(suggestion["common_names"]))
        if "url" in suggestion:
            print("More Info:", suggestion["url"])
        if "wiki_description" in suggestion and "value" in suggestion["wiki_description"]:
            print("Description:", suggestion["wiki_description"]["value"])
        print("\n")
else:
    print(f"Error: {response.status_code}, Message: {response.text}")


Possible plant identifications:
Species: Prunus avium, Probability: 25.10%


Species: Pyrus calleryana, Probability: 19.73%


Species: Prunus cerasus, Probability: 12.26%


Species: Prunus × yedoensis, Probability: 8.66%


Species: Magnolia, Probability: 3.21%


Species: Populus, Probability: 2.67%


Species: Dombeya, Probability: 2.57%


Species: Prunus × subhirtella, Probability: 2.14%


Species: Malus, Probability: 1.68%


Species: Prunus amygdalus, Probability: 1.67%




The message gives is a list of the possible candidates for the plant's ID, ordered by probability.
- We need to determine if we will choose the first one or allow the user to choose.
- We also need to determine how we will store this identity, so that we don't have to make the user identify the plant several times.

# Plant health


In [ ]:
import requests
import base64

# Define the endpoint and API key
url = "https://api.plant.id/v2/health_assessment"
api_key = userdata.get('plantid_key')  # Replace with your actual API key

# Load the image and encode it in base64
image_path = "plant.jpg"
with open(image_path, "rb") as image_file:
    base64_image = base64.b64encode(image_file.read()).decode("utf-8")

# Prepare headers and data payload
headers = {
    "Content-Type": "application/json",
    "Api-Key": api_key
}

data = {
    "images": [base64_image],
    "organs": ["leaf"],  # Optional: Specify the plant part (e.g., leaf, flower, etc.)
    "include-related-images": False  # Optional: Include related images in the response
}

# Send POST request
response = requests.post(url, headers=headers, json=data)



In [27]:
if response.status_code == 200:
    results = response.json()
    print("Plant Health Assessment Results:")

    # Check if the response contains health assessment information
    if "health_assessment" in results:
        health_info = results["health_assessment"]

        # Check if the plant is healthy
        if health_info["is_healthy"]:
            print(f"The plant is healthy with a probability of {health_info['is_healthy_probability']*100:.2f}%.")
        else:
            print("The plant shows signs of disease.")

            # If diseases are detected, print each disease and its details
            for disease in health_info["diseases"]:
                print(f"Diagnosis: {disease['name']}")
                print(f"Probability: {disease['probability']*100:.2f}%")
                print(f"Suggested Treatment: {disease.get('treatment', 'N/A')}")
                print("\n")
    else:
        print("No health assessment information available in the response.")
else:
    print(f"Error: {response.status_code}, Message: {response.text}")


Plant Health Assessment Results:
The plant is healthy with a probability of 94.42%.
